In [2]:
dataPath = 'C:\Physio Data\9.7.17\FZ_number4_8.1\p1_1'

In [3]:
import os
os.chdir(dataPath)
import sys
kwikToolsPath = 'C://Users/Lauren/Documents/Github/kwik-tools'
sys.path.append(kwikToolsPath)
import read_rhd as rhd
import numpy as np
import glob
import matplotlib.pyplot as plt
from shutil import copy2
%matplotlib inline

In [4]:
files = glob.glob('*.rhd')
for x in files:
    print(x)
files.sort(key=os.path.getmtime)  ## this sorting may be OS sensitive
print('\nSorted Files:')
for x in files:
    print(x)

airpuff_170907_162026.rhd
airpuff_170907_162125.rhd
airpuff_170907_162148.rhd
airpuff_170907_162247.rhd
airpuff_170907_162335.rhd
airpuff_170907_162434.rhd
airpuff_170907_162458.rhd
airpuff_170907_162557.rhd
airpuff_170907_162638.rhd
airpuff_170907_162737.rhd
airpuff_170907_162801.rhd
airpuff_170907_162900.rhd
airpuff_170907_163044.rhd
airpuff_170907_163158.rhd
airpuff_170907_163416.rhd
airpuff_170907_163522.rhd
airpuff_170907_163735.rhd
airpuff_170907_163954.rhd
airpuff_170907_164021.rhd
airpuff_170907_164048.rhd
airpuff_170907_164116.rhd
airpuff_170907_164143.rhd
airpuff_170907_164211.rhd
airpuff_170907_164238.rhd
airpuff_170907_164306.rhd
airpuff_170907_164333.rhd
airpuff_170907_164400.rhd
airpuff_170907_164428.rhd
airpuff_170907_164455.rhd
airpuff_170907_164523.rhd
airpuff_170907_164550.rhd
airpuff_170907_164618.rhd
airpuff_170907_164645.rhd
airpuff_170907_164712.rhd
airpuff_170907_164740.rhd
airpuff_170907_164807.rhd
airpuff_170907_164835.rhd
airpuff_170907_164902.rhd
airpuff_1709

In [6]:
basename = os.path.basename(dataPath) # taking foldername as basename

#batch write .rhd to .dat
for file in files:
    d = rhd.read_rhd(file)
    recordings = np.transpose(d['amplifier_data'])
    stimulation = ()
    #save amplifier data as .dat
    recordings.tofile(os.path.splitext(file)[0] +'.dat')  
    #save digital input streams
    for digitalChannel in range(d['board_dig_in_data'].shape[0]):
        d['board_dig_in_data'][digitalChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(digitalChannel) + '.di')
    #save analog input streams
    for analogChannel in range(d['aux_input_data'].shape[0]):
        d['aux_input_data'][analogChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(analogChannel) + '.ai')

#create .prm file and copy .prb file to data directory    
datFiles = glob.glob('*.dat')
datFiles.sort(key=os.path.getmtime)

prb_file = 'A1x32-Poly2-5mm-50s-177-A32.prb' #prb_file = 'Buzsaki32.prb'
copy2(kwikToolsPath+'/'+prb_file,dataPath)


with open(basename + '.prm', "w") as text_file:
        text_file.write('experiment_name = \'{0}\' \n'.format(basename))
        text_file.write('prb_file = \'{0}\'\n'.format(prb_file)) 
        text_file.write('traces = dict( \n')
        text_file.write('\traw_data_files={0},\n'.format(datFiles))
        text_file.write('\tvoltage_gain={0}.0,\n'.format(192))  #from Intan RHD2000 documentation
        text_file.write('\tsample_rate={0},\n'.format(d['frequency_parameters']['amplifier_sample_rate']))
        text_file.write('\tn_channels={0},\n'.format(len(d['amplifier_channels'])))
        text_file.write('\tdtype=\'uint16\'\n')
        text_file.write('\t)\n\n')
        
        text_file.write('spikedetekt = dict(\n')
        text_file.write('\tfilter_low=500.,\n') # low pass filter? (as documented in phy)
        text_file.write('\tfilter_high_factor=0.95 * .5,\n') # high pass filter (as documented in phy)
        text_file.write('\tfilter_butter_order=3,\n') # order of Butterworth Filter
        text_file.write('\n')
        text_file.write('\tfilter_lfp_low=20,\n') # LFP filter low-pass frequency
        text_file.write('\tfilter_lfp_high=0,\n') #LFP filter high-pass frequency
        text_file.write('\n')
        text_file.write('\tchunk_size_seconds=1,\n')
        text_file.write('\tchunk_overlap_seconds=.015,\n')
        text_file.write('\n')
        text_file.write('\tn_excerpts=50,\n')
        text_file.write('\texcerpts_size_seconds=1,\n')
        text_file.write('\tthreshold_strong_std_factor=5.5,\n')
        text_file.write('\tthreshold_weak_std_factor=3.5,\n')
        text_file.write("\tdetect_spikes='negative',\n")
        text_file.write('\n')
        text_file.write('\tconnected_component_join_size=1,\n')
        text_file.write('\n')
        text_file.write('\textract_s_before=16,\n')
        text_file.write('\textract_s_after=16,\n')
        text_file.write('\n')
        text_file.write('\tn_features_per_channel=3,\n') #number of features per channel
        text_file.write('\tpca_n_waveforms_max=10000,\n')
        text_file.write(')\n')
        text_file.write('\n')
        text_file.write('klustakwik2 = dict(\n')
        text_file.write('\tnum_starting_clusters=100,\n')
        text_file.write(')\n')
    




Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 59.976 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 3.5 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital ou

50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 14.6 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 1.080 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 0.1 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channe

40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 4.3 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 3.5 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply vo

10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 2.8 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 2.6 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxilia

80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 2.5 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 4.1 seconds

Reading

20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 2.5 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% d

10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 3.5 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% d

10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 3.4 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% d

10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 5.3 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% d

20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 5.7 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 4 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 26.064 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% d